In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from models import InferSent
import torch 
import json,os,gc
import pickle
from time import time
from copy import deepcopy
import nltk
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import wordnet,stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('all')
stopwords = set(stopwords.words('english'))

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import os,re,multiprocessing,joblib
from multiprocessing import Pool
from collections import defaultdict

number_of_articles_to_sample = 10000
first_selection = 750


path = 'C:\\Users\Ilias\\OneDrive\\Cours\\Semestre 07\\Linguistica\\practica_3\\'
path_text = path + 'pdf_json\\'

l_path = len(path_text)

file_names = []
temp_file_names = os.listdir(path_text)
file_names.extend([path_text+file_name for file_name in temp_file_names])
len(file_names)

MODEL_PATH = path+'infersent.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 2} #because we use infersent2 pre-trained model

infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = path+'crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

C:\Users\Ilias\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
path_text

'C:\\Users\\Ilias\\OneDrive\\Cours\\Semestre 07\\Linguistica\\practica_3\\pdf_json\\'

In [3]:
articles = np.random.choice(len(file_names),size=number_of_articles_to_sample,replace=False)
file_names = np.array(file_names)[articles]
len(file_names)

10000

In [4]:
#The glossary : a list of words that articles will contain, in order to keep articles of interest and to avoid pure-scientific ones
#using our corpus and this glossary, approximatively 20% og the texts will be accepted.
symptoms = [ #this one was taken from a notebook of the official kaggle competition https://www.kaggle.com/justmax1604/covid-text-extraction
    "weight loss","chills","shivering","convulsions","deformity","discharge","dizziness", "lymphopenia", "sneezing",
    "vertigo","fatigue","malaise","asthenia","hypothermia","jaundice","muscle weakness", "chest discomfort",
    "pyrexia","sweats","swelling","swollen","painful lymph node","weight gain","arrhythmia", "loss of smell", "loss of appetite", "loss of taste",
    "bradycardia","chest pain","claudication","palpitations","tachycardia","dry mouth","epistaxis", "dysgeusia", "hypersomnia", "taste loss",
    "halitosis","hearing loss","nasal discharge", "nasal inflammation", "otalgia","otorrhea","sore throat","toothache","tinnitus", "dysphonia",
    "trismus","abdominal pain","fever","bloating","belching","bleeding","bloody stool","melena","hematochezia", "burning sensation in the chest", 
    "constipation","diarrhea","dysphagia","dyspepsia","fecal incontinence","flatulence","heartburn", "chest tightness", "chest pressure",
    "nausea","odynophagia","proctalgia fugax","pyrosis","steatorrhea","vomiting","alopecia","hirsutism", "tachypnoea", "nasal obstruction",
    "hypertrichosis","abrasion","anasarca","bleeding into skin","petechia","purpura","ecchymosis", "bruising", 
    "blister","edema","itching","laceration","rash","urticaria","abnormal posturing","acalculia","agnosia","alexia",
    "amnesia","anomia","anosognosia","aphasia","apraxia","ataxia","cataplexy","confusion","dysarthria", "nasal congestion",
    "dysdiadochokinesia","dysgraphia","hallucination","headache","akinesia","bradykinesia","akathisia","athetosis",
    "ballismus","blepharospasm","chorea","dystonia","fasciculation","muscle cramps","myoclonus","opsoclonus","tic",
    "tremor","flapping tremor","insomnia","loss of consciousness","syncope","neck stiffness","opisthotonus",
    "paralysis","paresis","paresthesia","prosopagnosia","somnolence","abnormal vaginal bleeding", "neuralgia",
    "vaginal bleeding in early pregnancy", "miscarriage","vaginal bleeding in late pregnancy","amenorrhea", "body aches",
    "infertility","painful intercourse","pelvic pain","vaginal discharge","amaurosis fugax","amaurosis", "skin lesions",
    "blurred vision","double vision","exophthalmos","mydriasis","miosis","nystagmus","amusia","anhedonia",
    "anxiety","apathy","confabulation","depression","delusion","euphoria","homicidal ideation","irritability",
    "mania","paranoid ideation","suicidal ideation","apnea","hypopnea","cough","dyspnea","bradypnea","tachypnea",
    "orthopnea","platypnea","trepopnea","hemoptysis","pleuritic chest pain","sputum production","arthralgia",
    "back pain","sciatica","urologic","dysuria","hematospermia","hematuria","impotence","polyuria",
    "retrograde ejaculation","strangury","urethral discharge","urinary frequency","urinary incontinence","urinary retention", "anosmia", "myalgia", "rhinorrhea", "shortness of breath"]
text_file = open("glossary.txt", "r",encoding='utf-8')
other_terms = [i[:-1] for i in text_file.readlines()]
glossary = symptoms+other_terms

In [5]:
def file_content(file_path,gloss):
    n_pertinent_words=0
    abstract='';body_text = '';error_count = 0
    if os.path.splitext(file_path)[1]=='.json':
        f = open(file_path)
        f_json = json.load(f)
        title = f_json["metadata"]["title"]
        authors_json = f_json["metadata"]["authors"]
        authors = ''
        for i in authors_json:
            if (i['first']!='' or i['last']!=''):
                authors+=i['first']+' '+i['last']+', '
        if authors=='':
            authors='unknown'
        else:
            authors = authors[:-2]
        
        try:
            abstract = f_json['abstract'][0]['text']
        except:
            error_count+=1
        for i in f_json['body_text']:
            try:
                body_text= body_text+' '+i['text']
            except:
                error_count+=1
        body_text = body_text.strip()
        for word in gloss:
            if word in body_text:
                n_pertinent_words+=1
        f.close()
        return n_pertinent_words,body_text,abstract,authors,title,error_count
    else:
        return n_pertinent_words,body_text,abstract,authors,title,error_count


In [6]:
df = pd.DataFrame({'file_name':[],'article_no':[],'n_pertinent_words':[],'body':[],'abstract':[],'authors':[],'titles':[],'error_count':[]})
for ind in range(len(file_names)):
    df.loc[ind,'file_name']=file_names[ind][l_path:] #this value is to change
    df.loc[ind,'article_no']=ind
    df.loc[ind,'n_pertinent_words'],df.loc[ind,'body'],df.loc[ind,'abstract'],df.loc[ind,'authors'],df.loc[ind,'titles'],df.loc[ind,'error_count'] = file_content(file_path=file_names[ind],gloss=glossary)


In [7]:
df.drop(np.where(df.applymap(lambda x: x == ''))[0],inplace=True)
df.drop(np.where(pd.isnull(df))[0],inplace=True) #nan values,inplace=True)

In [8]:
df = df.iloc[np.argsort(df['n_pertinent_words'])[-first_selection:]]

In [9]:
# -- We take the 1000 documents (top 10%) that contain the most relevant words. 
# -- While varying the original corpus size, we could also use a treshold = np.quantile(df['n_pertinent_words'],0.8).
# -- The assumption made is that documents containing a lot of pertinent words will have more relevant content.
index = range(len(df))
df['Index'] = index
df['article_no'] = index
df = df.set_index(keys='Index',drop=True)
with open('documents.pickle', 'wb') as f:
    pickle.dump(df, f)

In [10]:
corpus_file = 'corpus.txt'
sent_dict_file = 'sent.joblib.compressed'
word_sent_no_dict_file = 'word_sent_no.joblib.compressed'
orig_word_sent_no_dict_file = 'orig_word_sent_no.joblib.compressed'
stopword_file = 'stopword.txt'
## Lemmatization function
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()
def get_lemmatize(sent):
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_tokenize(sent)])

def fn_lemmatize(data):
    for ind,info in tqdm(data.iterrows(),total=data.shape[0]):
        data.loc[ind,'sentence_lemmatized'] = get_lemmatize(sent = info['sentence'])
    return data
## removing stopwords

def words(text): 
    return re.findall(r'\w+', text.lower())


def remove_stopwords(sent):
    ## case conversion - lower case
    word_tokens = words(text=sent)
    #sent = sent.lower()
    #word_tokens = word_tokenize(sent)
    ## removing stopwords
    filtered_sentence = " ".join([w for w in word_tokens if not w in stopwords])
    ## removing digits
    filtered_sentence = re.sub(r'\d+','',filtered_sentence)
    ## removing multiple space
    filtered_sentence = words(text = filtered_sentence)
    return " ".join(filtered_sentence)

def fn_stopword(data):
    for ind,info in tqdm(data.iterrows(),total=data.shape[0]):
        sent = info['sentence_lemmatized']
        data.loc[ind,'sentence_lemma_stop'] = remove_stopwords(sent)
    return data

def fn_stopword_orig(data):
    for ind,info in tqdm(data.iterrows(),total=data.shape[0]):
        sent = info['sentence']
        data.loc[ind,'sentence_stop'] = remove_stopwords(sent)
    return data

In [11]:
## creating sentence dictionary
df['article'] = df['body']+' '+df['abstract']
df['article'].fillna('',inplace=True)
article_no_sent_dict = dict()
for ind,info in tqdm(df.iterrows(),total=df.shape[0]):
    sents = sent_tokenize(str(info['article']))
    for sent in sents: #removing lines that don't contain relevant word or that are too short. Will remove a bit more than 1/3 of the sents.
        is_ok=0
        for word in glossary:
            if word in sent:
                is_ok=1
        if is_ok==0 or len(sent)<25:
            sents.remove(sent)
    article_no_sent_dict[info['article_no']] = sents




100%|██████████| 750/750 [00:20<00:00, 36.19it/s]


In [12]:
article_no_list = list();sent_list = list()
df_sent = pd.DataFrame({'article_id':[],'sentence':[]})
for i in tqdm(article_no_sent_dict,total=len(article_no_sent_dict)):
    article_no_list.extend([i]*len(article_no_sent_dict[i]))
    sent_list.extend(article_no_sent_dict[i])
df_sent['article_id'] = article_no_list ; df_sent['sentence'] = sent_list
df_sent['sent_no'] = list(range(df_sent.shape[0]))
## sentence level dictionary
sent_dict = dict()
for ind,info in tqdm(df_sent.iterrows(),total=df_sent.shape[0]):
    sent_dict[info['sent_no']] = info['sentence']
sent_dict[-1] = 'NULL'
sent_dict_file = 'sent.joblib.compressed'
with open('document_sents.pickle', 'wb') as f:
    pickle.dump(df_sent, f)
joblib.dump(sent_dict,sent_dict_file, compress=True)

100%|██████████| 158465/158465 [00:17<00:00, 9209.14it/s] 


['sent.joblib.compressed']

In [13]:
outF = open(corpus_file, "w",encoding="utf-8")
for line in df_sent['sentence']:
    if len(line)>20:
 # write line to output file
        outF.write(line)
        outF.write("\n")
outF.close()

In [3]:


sentences = open(path+"corpus.txt", "r",encoding='utf-8').readlines()
len(sentences)

155635

In [15]:
#Build infersent embeddings : if they are not already made
infersent = InferSent({'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}) #because we use infersent2 pre-trained model
infersent.load_state_dict(torch.load(path+'infersent.pkl'))
infersent.set_w2v_path(path+'crawl-300d-2M.vec')
infersent.build_vocab(sentences, tokenize=True) #A few seconds. Vectorizes the vocabulary.
embeddings = infersent.encode(sentences, tokenize=True) #Up to a few hours with thousands of final-retained articles. Build vector representations of each words of each sentence.
joblib.dump(embeddings,'embeddings.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)
joblib.dump(infersent.word_vec,'word_vec.joblib',compress=True)#vectorization of our vocab

Found 84776(/139940) words with w2v vectors
Vocab size : 84776


['word_vec.joblib']

In [16]:
df_questions = pd.DataFrame({'questions':['Is covid a disease ?','Can we die because of covid ?'],'answers':['Yes','Yes']})
np.array(df_questions['questions'])
embeddings_questions = infersent.encode(np.array(df_questions['questions']), tokenize=True)
joblib.dump(embeddings_questions,'embeddings_questions.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)
joblib.dump(df_questions,'df_questions.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)


['df_questions.joblib']

In [15]:
def menu():
    print('Please enter your choice :\n\n')
    print('    1. Ask a question\n')
    print('    2. Add a question and its answer\n')
    print('    3. Informations\n')
    return(input('    4. Future updates\n'))
    

In [16]:
class InferSent_corpus:
    def __init__(self,path):
            
            self.infersent = InferSent({'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}) #because we use infersent2 pre-trained model
            self.infersent.load_state_dict(torch.load(path+'infersent.pkl'))
            self.infersent.set_w2v_path(path+'crawl-300d-2M.vec')
            self.embeddings = joblib.load(path+'embeddings.joblib')
            self.infersent.word_vec = joblib.load(path+'word_vec.joblib')

            self.sentences = open(path+"corpus.txt", "r",encoding='utf-8').readlines()
            with open(path+'documents.pickle', 'rb') as f:
                self.df = pickle.load(f)
            with open(path+'document_sents.pickle', 'rb') as f:
                self.df_sent = pickle.load(f)
                
    def find_similar(self,sent,n,time):
        embedded_query = self.infersent.encode([sent], tokenize=True) #query needs to be in an array
        similarity_matrix = cosine_similarity(embedded_query, self.embeddings)
        np.fill_diagonal(similarity_matrix, 0)
        similarities = similarity_matrix[0]
        if n == 1:
            return [np.argmax(similarities)]
        elif n is not None:
            return np.flip(similarities.argsort()[(-n*time):len(similarities)-(n*(time-1))][::1])

    def article_inf_from_sent(self,art_no):
        #sent=str(sent)
        #if sent[-1:]=='\n':
        #    sent = sent[:-1] #[:-1] we remove \n char due to encoding by lines
        
        article = self.df[self.df['article_no']==art_no]
        authors = article['authors']
        text = article['body']
        abstract = article['abstract']
        title = article['titles']
        return authors,text,abstract,title
    
    def proposal(self,sent,n,time=1):
        
        print(('\n Results of page '+str(time) + ' : \n'))
        for i in self.find_similar(sent,n,time):
            art_no = np.array(Corpus.df_sent[Corpus.df_sent['sentence'] == str(self.sentences[i])[:-1]]['article_id'])[0] #np.array to get the element as an itneger and not as a list
            print(('Article '+str(art_no)))
            print(self.sentences[i])
        value = input("To get more info on an article, enter its number. To see more results, enter 'More'. To make another search, enter 'Menu'. To go to previous page, enter 'Previous'.\n\n")
        if value=='More' or value=='more':
            self.proposal(sent,n,time+1)
            return()
        if value=='Menu' or value=='menu':
            menu()
            return()
        if value=='Previous' or value=='previous' or value=='p':
            if(time>1):
                self.proposal(sent,n,time-1)
                return()
            else:
                print('\nYou where to the first page. You will be sent to the menu.')
                menu()
                return()             

        try:
            self.print_infos(int(value))
        except:
            print('Error. You need to enter a valid option or number.\nYou were sent back to the menu.')
            menu()
        return()
    def print_infos(self,art_no):
        authors,text,abstract,title = self.article_inf_from_sent(art_no)
        print('\nArticle n°' + str(art_no) +'.\n' + 'Authors : ' + np.array(authors)[0] + '.\n' + 'Title : ' + np.array(title)[0] + '\n\n' + 'Abstract : ' + np.array(abstract)[0])
        value = input("\nTo read the article's body, enter 'read'. To make another search, enter 'menu'.\n\n")
        if value=='read' or value=='Read' or value=='r':
            print(np.array(text)[0] + '\n\n')
            menu()
            return()
        elif value=='Menu' or value=='menu':
            menu()
        else:
            print('Youre choice where not valid. You were sent back to the menu')
            menu
        return()
        

In [17]:
class Corpus_questions:

    def __init__(self,infersent_corpus):
        self.infersent_corpus=infersent_corpus
        self.embeddings_questions = joblib.load('embeddings_questions.joblib')
        self.df_questions = joblib.load('df_questions.joblib')
        
    def process_query(self,query):
        ind = self.find_similar_question(query,1)
        value = input('\nThis question is not recognized by the database.\nDid you mean : ' + np.array(df_questions['questions'].loc[ind])[0] + '(yes/no)\n')
        if value=='yes' or value=='y' or value=='Yes' or value=='Y':
            print('Answer : ' + np.array(df_questions['answers'].loc[ind])[0])
            menu()
            return()
        if value=='no' or value=='No' or value=='n' or value=='N':
            value = input('You could find your answer by exploring our search engine. Would you like to do it ? (yes/no)\n')
            if value=='no' or value=='No':
                print('You were sent to the menu\n\n')
                menu()
                return()
            if value=='yes' or value=='Yes' or value=='y':
                self.infersent_corpus.proposal(query,5)
                return()
            print('Invalid choice. You were sent to the menu\n\n')
            menu()
            return()

    def find_similar_question(self,sent,n):
            embedded_query = self.infersent_corpus.infersent.encode([sent], tokenize=True) #query needs to be in an array
            similarity_matrix = cosine_similarity(embedded_query, embeddings_questions)
            np.fill_diagonal(similarity_matrix, 0)
            similarities = similarity_matrix[0]
            if n == 1:
                return [np.argmax(similarities)]
            elif n is not None:
                return np.flip(similarities.argsort()[-n:][::1])

    def add_question_answer(self):
        #We won't check if the question is already entered. Actually, the more forms of the same question we have, the better it would be for InferSent.
        value = input('\nEnter your question.\n') 
        value2 = input('\nPlease enter your answer.\n')
        self.df_questions.loc[len(df_questions)] = list([value,value2])
        embedded_query = self.infersent_corpus.infersent.encode([value], tokenize=True)
        self.embeddings_questions = np.append(self.embeddings_questions,embedded_query,axis=0)
        print(self.embeddings_questions)
        joblib.dump(self.embeddings_questions,'embeddings_questions.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)
        joblib.dump(self.df_questions,'df_questions.joblib', compress=True) #Pickle can't read this variable so we use joblib (maybe too big...?)


        value = input('\n Thanks you very much. Would you like to add another question ? (yes/no)\n') 
        if value=='y' or value=='Y' or value=='Yes' or value=='yes':
            self.add_question_answer()
            return()
        else:
            print('\nYou were sent back to the menu.\n')
            #menu()
            return()


In [11]:
Corpus = InferSent_corpus(path)
Questions = Corpus_questions(Corpus)
print('-----Welcome in Sars-engine-----\n\n')
menu()

-----Welcome in Sars-engine-----


Please enter your choice :


    1. Ask a question

    2. Add a question and its answer

    3. Informations



    4. Future updates
 4


'4'

In [168]:
Questions.process_query('Salut')
Questions.add_question_answer()


 This question is not recognized by the database.
Did you mean : Can we die because of covid ?(yes/no)
 yes


Answer : Yes


()

In [157]:
Questions.add_question_answer()



Enter your question.
 yes

Please enter your answer.
 yes


[[ 0.00746889 -0.12430182 -0.04490045 ... -0.0378999  -0.01124218
  -0.03285642]
 [ 0.00746889 -0.03805197 -0.06995571 ... -0.01099967 -0.01057411
  -0.01039709]
 [ 0.00746889 -0.14438318 -0.06280056 ... -0.10446588 -0.06125115
  -0.04683772]]



 Thanks you very much. Would you like to add another question ? (yes/no)
 no



You were sent back to the menu.



()

In [ ]:
#define YES and NO tables (if value in YES,...)
#make articles reading readable
#forget pickle and use only joblib
#add weights for questions (if they are frecuently asked, they have more chance to be picked)
#add a research over article titles
#analyze influence of tokenize argument of infersent encoding
#find a way to delete model_path etc from first cell 
#find the way to add the vocabulary of the mqnual corpus's questions in the infersent vocab (not that much important)
#improve menu systems (always be able to go back, etc)
#prefilter infersent responses by ensuring important words are present in the sentences (or add weights). use nltk to get the most important words.